#  <div style="text-align: center"> Store sales forecasting </div>

###  <div style="text-align: center"> Thành viên nhóm </div>
|Họ và tên| MSSV  | 
|----------|-------------|
|Võ Quốc Huy|     20081001       | 
|  Nguyễn Quang Bảo       |     20083601      |   
|  Nguyễn Xuân Giang       |     20079601       |   
|  Ngô Hoàng Nhật Huy       |     20079071       |    


# Import các thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import calendar
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode(connected = True)
import dash
from dash import html,dcc

In [ ]:
colors={}
def colorFader(c1,c2,mix=0): 
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
c1='#FAA831' 
c2='#9A4800' 
n=9
for x in range(n+1):
    colors['level'+ str(n-x+1)] = colorFader(c1,c2,x/n) 
colors['background'] = '#232425'
colors['text'] = '#fff'


# Đọc dữ liệu từ file csv

In [ ]:
df_holi = pd.read_csv('data/holidays_events.csv')
df_oil = pd.read_csv('data/oil.csv')
df_stores = pd.read_csv('data/stores.csv')
df_trans = pd.read_csv('data/transactions.csv')
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

# Giới thiệu dữ liệu

Đây là bộ dữ liệu về doanh số bán hàng của cửa hàng Favorita ở Ecuador<br>
**Train.csv**<br>
    - Tập dữ liệu bao gồm chuỗi thời gian của các tính năng store_nbr , family và onpromotion cũng như doanh số mục tiêu .<br>
    - **store_nbr** xác định cửa hàng bán sản phẩm.<br>
    - **family** xác định loại sản phẩm được bán.<br>
        - **sales** cung cấp tổng doanh số cho một dòng sản phẩm tại một cửa hàng cụ thể vào một ngày nhất định. Giá trị phân số có thể thực hiện được vì các sản phẩm có thể được bán theo đơn vị phân số (ví dụ: 1,5 kg phô mai thay vì 1 túi khoai tây chiên).<br>
    - **onpromotion** cung cấp tổng số mặt hàng trong một dòng sản phẩm đang được khuyến mãi tại một cửa hàng vào một ngày nhất định.<br>
    
**Test.csv**<br>
    - Dữ liệu ở tập test này khá giống tập trai ở trên.<br>
    
**Stores.csv**<br>
    - Lưu trữ siêu dữ liệu, bao gồm: **city** , **state** , **type** và **cluter**.<br>
    - **cluter** là một nhóm các cửa hàng tương tự.<br>

**oil.csv**<br>
    - Giá dầu hàng ngày. Bao gồm các giá trị trong cả khung thời gian train và test dữ liệu. (Ecuador là một quốc gia phụ thuộc vào dầu mỏ và nền kinh tế của nước này rất dễ bị tổn thương trước những cú sốc về giá dầu.)<br>
    
**holidays_events.csv**<br>
    - Tập dữ liệu bao gồm các ngày lễ và sự kiện ở Ecuador<br>
    
**transactions.csv**<br>
    - Ngày giao dịch tại các cửa hàng

# Gộp dữ liệu

In [ ]:
df_train1 = df_train.merge(df_holi, on = 'date', how='left')
df_train1 = df_train1.merge(df_oil, on = 'date', how='left')
df_train1 = df_train1.merge(df_stores, on = 'store_nbr', how='left')
df_train1 = df_train1.merge(df_trans, on = ['date', 'store_nbr'], how='left')
df_train1 = df_train1.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})

df_train1['date'] = pd.to_datetime(df_train1['date'])
df_train1['year'] = df_train1['date'].dt.year
df_train1['month'] = df_train1['date'].dt.month
df_train1['week'] = df_train1['date'].dt.isocalendar().week
df_train1['quarter'] = df_train1['date'].dt.quarter
df_train1['day_of_week'] = df_train1['date'].dt.day_name()
df_train1.head()

# Làm sạch dữ liệu

In [ ]:
df_train1.shape

In [ ]:
df_train1.dropna(inplace=True)

In [ ]:
df_train1.shape

In [ ]:
df_train1.head()

# Khám phá dữ liệu

## Phân tích doanh thu trung bình của cửa hàng 
### Top 10 sản phẩm bán chạy nhất 

In [ ]:
df_fa_sa = df_train1.groupby('family').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_fa_sa['color'] = colors['level10']
df_fa_sa['color'][:1] = colors['level1']
df_fa_sa['color'][1:2] = colors['level2']
df_fa_sa['color'][2:3] = colors['level3']
df_fa_sa['color'][3:4] = colors['level4']
df_fa_sa['color'][4:5] = colors['level5']

fig1 = go.Figure(data=[go.Bar(x=df_fa_sa['sales'],
                             y=df_fa_sa['family'], 
                             marker=dict(color= df_fa_sa['color']),
                             name='Family', orientation='h',
                             text=df_fa_sa['sales'].astype(int),
                             textposition='auto',
                             hoverinfo='text',
                             hovertext=
                            '<b>Family</b>:'+ df_fa_sa['family'] +'<br>' +
                            '<b>Sales</b>:'+ df_fa_sa['sales'].astype(int).astype(str) +'<br>' ,
                            # hovertemplate='Family: %{y}'+'<br>Sales: $%{x:.0f}'
                            )])
fig1.update_layout(title_text='Top 10 sản phẩm bán chạy nhất ',paper_bgcolor=colors['background'],plot_bgcolor=colors['background'],
                font=dict(
                size=14,
                color='white'))

fig1.update_yaxes(showgrid=False, categoryorder='total ascending')



**Sản phẩm bán chạy nhất là: GROCREY và BEVERAGES**

### Doanh số bán hàng trung bình với loại cửa hàng

In [ ]:
df_st_sa = df_train1.groupby('store_type').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
fig2=go.Figure(data=[go.Pie(values=df_st_sa['sales'], labels=df_st_sa['store_type'], name='Store type',
                     marker=dict(colors=[colors['level1'],colors['level3'],colors['level5'],colors['level7'],colors['level9']]), hole=0.7,
                     hoverinfo='label+percent+value', textinfo='label'
                    )])
fig2.update_layout(title_text='Doanh số bán hàng trung bình với loại cửa hàng',paper_bgcolor=colors['background'],plot_bgcolor='#1f2c56',
                font=dict(
                size=14,
                color='white'))
fig2.update_yaxes(showgrid=False, categoryorder='total ascending')

**Cửa hàng loại A có doanh số bán hàng cao nhất với 38%.**

### Cụm cửa hàng và doanh số bán hàng

In [ ]:
df_cl_sa = df_train1.groupby('cluster').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_cl_sa['color'] = colors['level10']
df_cl_sa['color'][:1] = colors['level1']
df_cl_sa['color'][1:2] = colors['level2']
df_cl_sa['color'][2:3] = colors['level3']
df_cl_sa['color'][3:4] = colors['level4']
df_cl_sa['color'][4:5] = colors['level5']
fig3 = go.Figure(data=[go.Bar(y=df_cl_sa['sales'],
                             x=df_cl_sa['cluster'], 
                             marker=dict(color= df_cl_sa['color']),
                             name='Cluster',
                             text=df_cl_sa['sales'].astype(int),
                             textposition='auto',
                             hoverinfo='text',
                             hovertext=
                            '<b>Cluster</b>:'+ df_cl_sa['cluster'].astype(str) +'<br>' +
                            '<b>Sales</b>:'+ df_cl_sa['sales'].astype(int).astype(str) +'<br>' ,
                            # hovertemplate='Family: %{y}'+'<br>Sales: $%{x:.0f}'
                            )])
fig3.update_layout(title_text='Clusters Vs Sales',paper_bgcolor=colors['background'],plot_bgcolor=colors['background'],
                font=dict(
                size=14,
                color='white'))

fig3.update_xaxes(tickmode = 'array', tickvals=df_cl_sa.cluster)
fig3.update_yaxes(showgrid=False)

**Cụm 5 có doanh số bán hàng cao nhất**

### Doanh số bán hàng trung bình giữa các thành phố

In [ ]:
df_city_sa = df_train1.groupby('city').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_city_sa['color'] = colors['level10']
df_city_sa['color'][:1] = colors['level1']
df_city_sa['color'][1:2] = colors['level2']
df_city_sa['color'][2:3] = colors['level3']
df_city_sa['color'][3:4] = colors['level4']
df_city_sa['color'][4:5] = colors['level5']

fig4 = go.Figure(data=[go.Bar(y=df_city_sa['sales'],
                             x=df_city_sa['city'], 
                             marker=dict(color= df_city_sa['color']),
                             name='State',
                             text=df_city_sa['sales'].astype(int),
                             textposition='auto',
                             hoverinfo='text',
                             hovertext=
                            '<b>City</b>:'+ df_city_sa['city'] +'<br>' +
                            '<b>Sales</b>:'+ df_city_sa['sales'].astype(int).astype(str) +'<br>' ,
                            # hovertemplate='Family: %{y}'+'<br>Sales: $%{x:.0f}'
                            )])
fig4.update_layout(title_text='Doanh số bán hàng trung bình giữa các thành phố',paper_bgcolor=colors['background'],plot_bgcolor=colors['background'],
                font=dict(
                size=14,
                color='white'))

fig4.update_yaxes(showgrid=False, categoryorder='total ascending')

**2 thành phố có doanh thu trung bình cao nhất là Quito và Cayambe**

### Top 5 khu vực có doanh số bán hàng trung bình cao nhất

In [ ]:
df_state_sa = df_train1.groupby('state').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:5]
df_state_sa['color'] = colors['level10']
df_state_sa['color'][:1] = colors['level1']
df_state_sa['color'][1:2] = colors['level2']
df_state_sa['color'][2:3] = colors['level3']
df_state_sa['color'][3:4] = colors['level4']
df_state_sa['color'][4:5] = colors['level5']
df_state_sa
fig5 = go.Figure(data=[go.Bar(y=df_state_sa['sales'],
                             x=df_state_sa['state'], 
                             marker=dict(color= df_state_sa['color']),
                             name='State',
                             text=df_state_sa['sales'].astype(int),
                             textposition='auto',
                             hoverinfo='text',
                             hovertext=
                            '<b>State</b>:'+ df_state_sa['state'] +'<br>' +
                            '<b>Sales</b>:'+ df_state_sa['sales'].astype(int).astype(str) +'<br>' ,
                            # hovertemplate='Family: %{y}'+'<br>Sales: $%{x:.0f}'
                            )])
fig5.update_layout(title_text='Top 5 doanh số bán hàng trung bình cao nhất giữa các khu vực',paper_bgcolor=colors['background'],plot_bgcolor=colors['background'],
                font=dict(
                size=14,
                color='white'))

fig5.update_yaxes(showgrid=False, categoryorder='total ascending')

### Doanh số trung bình hàng ngày

In [ ]:
df_day_sa = df_train1.groupby('date').agg({"sales" : "mean"}).reset_index()
fig6 = go.Figure(data=[go.Scatter(x=df_day_sa['date'], y=df_day_sa['sales'], fill='tozeroy', fillcolor='#FAA831', line_color='#bA6800'                                 )])
fig6.update_layout(title_text='Doanh số trung bình hằng ngày',height=300,paper_bgcolor='#232425',plot_bgcolor='#232425',
                font=dict(
                size=12,
                color='white'))
fig6.update_xaxes(showgrid=False)
fig6.update_yaxes(showgrid=False)


### Doanh số trung bình hàng tuần

In [ ]:
df_w_sa = df_train1.groupby('week').agg({"sales" : "mean"}).reset_index()
fig7 = go.Figure(data=[go.Scatter(x=df_w_sa['week'], y=df_w_sa['sales'], fill='tozeroy', fillcolor='#FAA831', line_color='#bA6800'
                                  ,mode='lines+markers')])


fig7.update_layout(title_text='Doanh số trung bình hàng tuần',height=300,paper_bgcolor='#232425',plot_bgcolor='#232425',
                font=dict(
                size=12,
                color='white'))
fig7.update_yaxes(showgrid=False)
fig7.update_xaxes(showgrid=False,tickmode = 'array', tickvals=df_w_sa.week, ticktext=[i for i in range(1,53)])

### Doanh số trung bình hàng tháng

In [ ]:
df_mon_sa = df_train1.groupby('month').agg({"sales" : "mean"}).reset_index()
fig8 = go.Figure(data=[go.Scatter(x=df_mon_sa['month'], y=df_mon_sa['sales'], fill='tozeroy', fillcolor='#FAA831', line_color='#bA6800'
                                  ,mode='lines+markers')])


fig8.update_layout(title_text='Doanh số trung bình hàng tháng',height=300,paper_bgcolor='#232425',plot_bgcolor='#232425',
                font=dict(
                size=12,
                color='white'))
fig8.update_yaxes(showgrid=False)
fig8.update_xaxes(showgrid=False,tickmode = 'array', tickvals=df_mon_sa.month)

### Khu vực và thành phố

In [ ]:
df_c_s_sa = df_train1.groupby(['state','city']).agg({"sales" : "mean"}).reset_index()
df_c_s_sa=df_c_s_sa[df_c_s_sa.sales>0]
fig9 = px.sunburst(df_c_s_sa, path=['state', 'city' ], 
                    values='sales',color='sales',
                    color_continuous_scale=[colors['level1'], colors['level10']])

fig9.update_layout(title_text='States & Cities',width = 700,paper_bgcolor='#232425',plot_bgcolor='#232425',font=dict(color=colors['text']))
fig9.show()

### Doanh số trung bình giữa loại cửa hàng và loại kì nghỉ

In [ ]:
df_st_ht = df_train1.groupby(['store_type','holiday_type']).agg({"sales" : "mean"}).reset_index()
df_st_ht['sales'] = round(df_st_ht['sales'], 2)

fig10 = px.scatter(df_st_ht, x='store_type', color='sales', y='holiday_type', size='sales',
                 color_discrete_sequence=px.colors.qualitative.D3,
                 title="Average Sales: Store Type Vs Holiday Type")
fig10.update_yaxes(ticksuffix='  ')
fig10.update_layout(height=400, xaxis_title='', yaxis_title='',
                  margin=dict(b=0),
                  plot_bgcolor='#232425', paper_bgcolor='#232425',
                  title_font=dict(size=29, color='#fafafa', family="Lato, sans-serif"),
                  font=dict(color='#fafafa'), 
                  hoverlabel=dict(bgcolor="#fafafa", font_size=13, font_family="Lato, sans-serif"))
fig10.show()

**Các cửa hàng loại A luôn có doanh số bán hàng cao hơn các loại còn lại**

### Doanh số trung bình giữa các tháng và loại kì nghỉ

In [ ]:
df_m_ht = df_train1.groupby(['month','holiday_type']).agg({"sales" : "mean"}).reset_index()
df_m_ht['sales'] = round(df_m_ht['sales'], 2)

fig11 = px.scatter(df_m_ht, x='month', color='sales', y='holiday_type', size='sales',
                 color_discrete_sequence=px.colors.qualitative.D3,
                 title="Doanh số trung bình giữa các tháng và loại kì nghỉ")

fig11.update_yaxes(ticksuffix='  ')
fig11.update_xaxes(tickmode = 'array', tickvals=[i for i in range(1,13)], 
                 ticktext=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
fig11.update_layout(height=400, xaxis_title='', yaxis_title='',
                  margin=dict(b=0),
                  plot_bgcolor='#232425', paper_bgcolor='#232425',
                  title_font=dict(size=29, color='#fafafa', family="Lato, sans-serif"),
                  font=dict(color='#fafafa'), 
                  hoverlabel=dict(bgcolor="#fafafa", font_size=13, font_family="Lato, sans-serif"))
fig11.show()

**Doanh số bán hàng cao ở các tháng 11, 12, 1, 5**

### Doanh số trung bình giữa các năm và loại kì nghỉ

In [ ]:
df_y_m_ht = df_train1.groupby(['year','month','holiday_type']).agg({"sales" : "mean"}).reset_index()
df_y_m_ht['sales'] = round(df_y_m_ht['sales'], 2)

fig12 = px.scatter(df_y_m_ht, x='month', y='holiday_type', color='sales', size='sales', 
                 facet_row='year', title='Doanh số trung bình giữa các năm và loại kì nghỉ')

fig12.update_yaxes(ticksuffix='  ')
fig12.update_xaxes(tickmode = 'array', tickvals=[i for i in range(1,13)], 
                 ticktext=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
fig12.update_layout(height=900, xaxis_title='', yaxis_title='',
                  margin=dict(t=70, b=0),
                  plot_bgcolor='#232425', paper_bgcolor='#232425',
                  title_font=dict(size=29, color='#fafafa', family="Lato, sans-serif"),
                  font=dict(color='#fafafa'), 
                  hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"))
fig12.show()

**Nhu cầu mua hàng qua các năm và tháng ngày càng tăng cao**

# Dashboard

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

graph1 = dcc.Graph(
        id='fig1',
        figure=fig1,
        className="six columns" 
    )
graph2 = dcc.Graph(
        id='fig2',
        figure=fig2,
        className="six columns"
    )
graph3 = dcc.Graph(
        id='fig3',
        figure=fig3,
        className="six columns"
    )
graph4 = dcc.Graph(
        id='fig4',
        figure=fig4,
        className="six columns"
    )
graph5 = dcc.Graph(
        id='fig5',
        figure=fig5,
        className="six columns"
    )
graph6 = dcc.Graph(
        id='fig6',
        figure=fig6,
        className="twelve columns"
    )
graph7 = dcc.Graph(
        id='fig7',
        figure=fig7,
        className="twelve columns"
    )
graph8 = dcc.Graph(
        id='fig8',
        figure=fig8,
        className="twelve columns"
    )
graph9 = dcc.Graph(
        id='fig9',
        figure=fig9,
        className="six columns"
    )
graph10 = dcc.Graph(
        id='fig10',
        figure=fig10,
        className="six columns"
    )
graph11 = dcc.Graph(
        id='fig11',
        figure=fig11,
        className="six columns"
    )  
graph12 = dcc.Graph(
        id='fig12',
        figure=fig12,
        className="twelve columns"
    )   

header = html.H2(children="Store sales forecasting",className="card_container")

row1 = html.Div(children=[graph1, graph5])
row2 = html.Div(children=[graph3, graph4])
row3 = html.Div(children=[graph2, graph9])
row7 = html.Div(children=[graph10, graph11])
row4 = html.Div(children=[graph6])
row5 = html.Div(children=[graph7])
row6 = html.Div(children=[graph8])
row8 = html.Div(children=[graph12])

layout = html.Div(children=[header, row1, row2,row3,row7,row4,row5,row6,row8], style={"text-align": "center"})
app.layout = layout   


if __name__ == "__main__":
    app.run_server()